In [1]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json

load_dotenv(override=True)

openai_api_key= os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"Key starts with {openai_api_key[:10]}")
else:
    print("Key doesnt exists.")

openai = OpenAI() 
# System prompt
system_message = "You are a helpful assistant that can check flight prices, weather, and currency conversions."

Key starts with sk-proj-Mf


In [2]:
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499"
}

weather_data = {
    "paris": "Sunny, 22°C",
    "tokyo": "Cloudy, 18°C",
    "london": "Rainy, 14°C",
    "berlin": "Windy, 15°C"
}

exchange_rates = {
    ("USD", "EUR"): 0.93,
    ("EUR", "USD"): 1.07,
    ("USD", "JPY"): 153.2,
    ("JPY", "USD"): 0.0065
}

In [3]:
# Tool functions
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")
    
def get_weather(destination_city):
    print(f"Tool get_weather called for {destination_city}")
    city = destination_city.lower()
    return weather_data.get(city, "Unknown")  

def get_exchange_rate(from_currency, to_currency):
    print(f"Tool get_exchange_rate called: {from_currency} → {to_currency}")
    return exchange_rates.get((from_currency.upper(), to_currency.upper()), "Unknown")    

In [4]:
# === TOOL FUNCTION MAP ===
function_map = ({
    "get_ticket_price": get_ticket_price,
    "get_weather": get_weather,
    "get_exchange_rate": get_exchange_rate,})

# === TOOL DEFINITIONS ===
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the ticket price for a specific destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {"type": "string", "description": "City to get the ticket price for"             }
                },
                "required": ["destination_city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {"type": "string"}
                },
                "required": ["destination_city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate",
            "description": "Get exchange rate between two currencies.",
            "parameters": {
                "type": "object",
                "properties": {
                    "from_currency": {"type": "string"},
                    "to_currency": {"type": "string"}
                },
                "required": ["from_currency", "to_currency"]
            }
        }
    }
]

In [6]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    func_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if func_name not in function_map:
        raise ValueError(f"Function {func_name} not implemented.")

    result = function_map[func_name](**arguments)

    response = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result if isinstance(result, dict) else {"result": result})
    }
    return response

def handle_tool_calls(tool_calls):
    responses = []
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        result = function_map[func_name](**arguments)
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result if isinstance(result, dict) else {"result": result})
        }
        responses.append(response)
    return responses

# Main chat function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages,tools=tools)
    choice = response.choices[0]

    # If a tool was called
    if choice.finish_reason == "tool_calls":
        tool_messages = choice.message
        tool_responses = handle_tool_calls(tool_messages.tool_calls)
        messages.append(tool_messages)
        messages.extend(tool_responses)
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

    return response.choices[0].message.content


In [7]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_weather called for London
Tool get_exchange_rate called: EUR → USD
